# Bundesliga Odds Test

In [ ]:
from sports.futbol.bundesliga.bundesliga_config import get_bundesliga_odds_config
from services.odds import OddsService

config = get_bundesliga_odds_config()
service = OddsService(sport='bundesliga', config=config)

In [ ]:
# Fetch schedule
schedule = service.fetch_schedule()
print(f"Found {len(schedule)} games")
for game in schedule[:5]:  # Show first 5
    print(f"  {game.get('matchup')} - Event ID: {game.get('event_id')}")

In [ ]:
# Fetch odds for first game
if schedule:
    first_game = schedule[0]
    event_id = first_game.get('event_id')
    print(f"Fetching odds for: {first_game.get('matchup')} (Event ID: {event_id})")
    
    odds_data = service.scraper.fetch_odds_from_api(event_id)
    print(f"\nTeams: {odds_data.get('teams')}")
    print(f"Game Date: {odds_data.get('game_date')}")

In [ ]:
# Show game lines
print("Game Lines:")
game_lines = odds_data.get('game_lines', {})
for market, data in game_lines.items():
    print(f"  {market}: {data}")

In [ ]:
# Show player props
print(f"\nPlayer Props: {len(odds_data.get('player_props', []))} players")
for player in odds_data.get('player_props', [])[:10]:  # First 10
    print(f"  {player.get('player')} ({player.get('team')})")
    for prop in player.get('props', []):
        if 'milestones' in prop:
            print(f"    - {prop.get('market')}: {len(prop.get('milestones', []))} lines")
        else:
            print(f"    - {prop.get('market')}: {prop.get('odds')}")

In [ ]:
# Save odds to verify full pipeline
path = service.save_odds(odds_data)
print(f"\nSaved odds to: {path}")

# Check what files were created
import os
for f in os.listdir(path):
    print(f"  - {f}")

In [ ]:
# Show CSV content
import pandas as pd
props_df = pd.read_csv(path / 'player_props.csv')
print(f"Player props: {len(props_df)} rows")
print(f"Markets: {props_df['market'].unique().tolist()}")
props_df.head(20)